In [286]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np

In [287]:
train = pd.read_csv("/content/drive/MyDrive/train.csv")
test = pd.read_csv('/content/drive/MyDrive/test.csv')
gender = pd.read_csv("/content/drive/MyDrive/gender_submission.csv")

In [288]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [289]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [290]:
train.shape

(891, 12)

In [291]:
test.shape

(418, 11)

In [292]:
train.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [293]:
test.isnull().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Cabin,327


In [294]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


*Filling or Cleanning Data*

In [295]:
df = [train, test]
for data in df:
    data['Age'] = data.groupby(['Pclass', 'Sex'])['Age'].transform(
        lambda x: x.fillna(x.median())
    )
    # dropping items
train = train.drop('Cabin',axis=1)
test = test.drop('Cabin',axis=1)
# filling with mode
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])
test['Fare'] = test['Fare'].fillna(test['Fare'].mode()[0])

In [296]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          418 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         418 non-null    float64
 9   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 32.8+ KB


**Outliers Detection**

In [297]:
num_cols = train.select_dtypes(include=['int64','float64'])

Q1 = num_cols.quantile(0.25)
Q3 = num_cols.quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = (num_cols < lower_bound) | (num_cols > upper_bound)
train = train[~outliers.any(axis=1)]

In [298]:
num_cols = test.select_dtypes(include=['int64','float64'])

Q1 = num_cols.quantile(0.25)
Q3 = num_cols.quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = (num_cols < lower_bound) | (num_cols > upper_bound)
test = test[~outliers.any(axis=1)]

In [299]:
for col in [train,test]:
  col['Sex'] = col['Sex'].map({"male":0,"female":1})

In [300]:
train = pd.get_dummies(train,columns=['Embarked'],drop_first=True)
test = pd.get_dummies(test,columns=['Embarked'],drop_first=True)

In [301]:
import re

for data in [train, test]:
    data['Title'] = data['Name'].apply(lambda x: re.search(r',\s*([^.]*)\.', x).group(1).strip())

# Group rare titles
for data in [train, test]:
    data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt','Col', 'Don', 'Dr', 'Major',
                                           'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Ms', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')

# One-hot encode Title
full = pd.concat([train, test])
full = pd.get_dummies(full, columns=['Title'], drop_first=True)

# Drop Name
full.drop('Name', axis=1, inplace=True)

# Split back
train = full.iloc[:len(train)]
test = full.iloc[len(train):]


In [302]:
for col in [train,test]:
  bool_cats =col.select_dtypes(include='bool').columns
  col[bool_cats]= col[bool_cats].astype(int)

/tmp/ipython-input-302-3716754861.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[bool_cats]= col[bool_cats].astype(int)
/tmp/ipython-input-302-3716754861.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[bool_cats]= col[bool_cats].astype(int)


In [303]:
train = train.drop('Ticket',axis=1)
test = test.drop('Ticket',axis=1)

In [304]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286 entries, 0 to 416
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  286 non-null    int64  
 1   Survived     0 non-null      float64
 2   Pclass       286 non-null    int64  
 3   Sex          286 non-null    int64  
 4   Age          286 non-null    float64
 5   SibSp        286 non-null    int64  
 6   Parch        286 non-null    int64  
 7   Fare         286 non-null    float64
 8   Embarked_Q   286 non-null    int64  
 9   Embarked_S   286 non-null    int64  
 10  Title_Miss   286 non-null    int64  
 11  Title_Mr     286 non-null    int64  
 12  Title_Mrs    286 non-null    int64  
 13  Title_Rare   286 non-null    int64  
dtypes: float64(3), int64(11)
memory usage: 33.5 KB


**Spliting modell for Random Forest**

In [305]:
x_train_spl = train.drop('Survived',axis=1)
y_train_spl = train['Survived']
# x_test = test.drop('Survived',axis=1)
# y_test = test['Survived']

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_train_spl,y_train_spl,test_size=0.2,random_state=42,stratify=y_train_spl)


**Random Forest Training**

In [306]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import   accuracy_score, classification_report, confusion_matrix,precision_score, recall_score, f1_score

In [307]:
model = RandomForestClassifier(n_estimators=100,random_state=42,class_weight='balanced')
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

In [308]:
print(f"Accuracy Score🟨🟨2️⃣: {accuracy_score(y_test,y_pred)}")
print(f"Classification Report🟨🟨2️⃣: {classification_report(y_test,y_pred)}")
print(f"Confusion matrix🟨🟨2️⃣: {confusion_matrix(y_test,y_pred)}")
print(f"precision score🟨🟨2️⃣: {precision_score(y_test,y_pred)}")
print(f"recall score🟨🟨2️⃣: {recall_score(y_test,y_pred)}")
print(f"f1 score🟨🟨2️⃣: {f1_score(y_test,y_pred)}")

Accuracy Score🟨🟨2️⃣: 0.8205128205128205
Classification Report🟨🟨2️⃣:               precision    recall  f1-score   support

         0.0       0.86      0.89      0.87        82
         1.0       0.72      0.66      0.69        35

    accuracy                           0.82       117
   macro avg       0.79      0.77      0.78       117
weighted avg       0.82      0.82      0.82       117

Confusion matrix🟨🟨2️⃣: [[73  9]
 [12 23]]
precision score🟨🟨2️⃣: 0.71875
recall score🟨🟨2️⃣: 0.6571428571428571
f1 score🟨🟨2️⃣: 0.6865671641791045


**XGBoost Testing**

In [309]:
import xgboost as xgb

In [314]:
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

xgb_model.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:02:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [315]:
y_pred = xgb_model.predict(x_test)

print(f"✅ Accuracy Score: {accuracy_score(y_test, y_pred)}")
print(f"\n✅ Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")
print(f"\n✅ Classification Report:\n{classification_report(y_test, y_pred)}")
print(f"✅ Precision: {precision_score(y_test, y_pred)}")
print(f"✅ Recall: {recall_score(y_test, y_pred)}")
print(f"✅ F1 Score: {f1_score(y_test, y_pred)}")


✅ Accuracy Score: 0.8547008547008547

✅ Confusion Matrix:
[[77  5]
 [12 23]]

✅ Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      0.94      0.90        82
         1.0       0.82      0.66      0.73        35

    accuracy                           0.85       117
   macro avg       0.84      0.80      0.82       117
weighted avg       0.85      0.85      0.85       117

✅ Precision: 0.8214285714285714
✅ Recall: 0.6571428571428571
✅ F1 Score: 0.7301587301587301
